# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [42]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

#https://www.technologyreview.com/2018/02/21/145289/the-ganfather-the-man-whos-given-machines-the-gift-of-imagination/

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [43]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [33]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

#enumerate return a tuple, for index number and item at that index, that tuple is taken as input to create a dict,
#and we iterate over the entire 'char' which here is a list/tuple containg all unique characters.

int2char = dict(enumerate(chars))

#now taking the formed dict, we make chacter : index pairs by storing it in a dict in rev order when we extract 
# (index,char) tuples from the dict int2char

char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
print(chars,'\n')
print('Number of unique characters : ',len(chars))

('W', '/', ':', '"', 'M', 'T', 'n', 'f', 'I', '3', 'p', 'J', '(', 'B', 'O', 'A', '2', 'N', '4', 'w', 'Y', 'k', '$', '!', '1', 'D', 'Z', 'i', 'b', 'j', '\n', 's', 'H', ';', 'G', 'P', '@', 'Q', 'a', 'm', "'", 'E', '&', '%', 'x', 'r', '7', 'l', 'u', 'F', 'X', ' ', '.', 'z', '8', '?', '*', 'V', 'o', 'q', 'K', '6', 'c', 'e', 'S', 'C', 'L', ',', '-', ')', 'v', 'd', 't', '9', '_', 'R', 'U', 'g', 'y', '5', '0', '`', 'h') 

Number of unique characters :  83


In [34]:
int2char

{0: 'W',
 1: '/',
 2: ':',
 3: '"',
 4: 'M',
 5: 'T',
 6: 'n',
 7: 'f',
 8: 'I',
 9: '3',
 10: 'p',
 11: 'J',
 12: '(',
 13: 'B',
 14: 'O',
 15: 'A',
 16: '2',
 17: 'N',
 18: '4',
 19: 'w',
 20: 'Y',
 21: 'k',
 22: '$',
 23: '!',
 24: '1',
 25: 'D',
 26: 'Z',
 27: 'i',
 28: 'b',
 29: 'j',
 30: '\n',
 31: 's',
 32: 'H',
 33: ';',
 34: 'G',
 35: 'P',
 36: '@',
 37: 'Q',
 38: 'a',
 39: 'm',
 40: "'",
 41: 'E',
 42: '&',
 43: '%',
 44: 'x',
 45: 'r',
 46: '7',
 47: 'l',
 48: 'u',
 49: 'F',
 50: 'X',
 51: ' ',
 52: '.',
 53: 'z',
 54: '8',
 55: '?',
 56: '*',
 57: 'V',
 58: 'o',
 59: 'q',
 60: 'K',
 61: '6',
 62: 'c',
 63: 'e',
 64: 'S',
 65: 'C',
 66: 'L',
 67: ',',
 68: '-',
 69: ')',
 70: 'v',
 71: 'd',
 72: 't',
 73: '9',
 74: '_',
 75: 'R',
 76: 'U',
 77: 'g',
 78: 'y',
 79: '5',
 80: '0',
 81: '`',
 82: 'h'}

In [7]:
char2int

{'W': 0,
 '/': 1,
 ':': 2,
 '"': 3,
 'M': 4,
 'T': 5,
 'n': 6,
 'f': 7,
 'I': 8,
 '3': 9,
 'p': 10,
 'J': 11,
 '(': 12,
 'B': 13,
 'O': 14,
 'A': 15,
 '2': 16,
 'N': 17,
 '4': 18,
 'w': 19,
 'Y': 20,
 'k': 21,
 '$': 22,
 '!': 23,
 '1': 24,
 'D': 25,
 'Z': 26,
 'i': 27,
 'b': 28,
 'j': 29,
 '\n': 30,
 's': 31,
 'H': 32,
 ';': 33,
 'G': 34,
 'P': 35,
 '@': 36,
 'Q': 37,
 'a': 38,
 'm': 39,
 "'": 40,
 'E': 41,
 '&': 42,
 '%': 43,
 'x': 44,
 'r': 45,
 '7': 46,
 'l': 47,
 'u': 48,
 'F': 49,
 'X': 50,
 ' ': 51,
 '.': 52,
 'z': 53,
 '8': 54,
 '?': 55,
 '*': 56,
 'V': 57,
 'o': 58,
 'q': 59,
 'K': 60,
 '6': 61,
 'c': 62,
 'e': 63,
 'S': 64,
 'C': 65,
 'L': 66,
 ',': 67,
 '-': 68,
 ')': 69,
 'v': 70,
 'd': 71,
 't': 72,
 '9': 73,
 '_': 74,
 'R': 75,
 'U': 76,
 'g': 77,
 'y': 78,
 '5': 79,
 '0': 80,
 '`': 81,
 'h': 82}

In [8]:
#first 100 characters in the book
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [9]:
# encoded for of forst 100 characters in the book
encoded[:100]

array([65, 82, 38, 10, 72, 63, 45, 51, 24, 30, 30, 30, 32, 38, 10, 10, 78,
       51,  7, 38, 39, 27, 47, 27, 63, 31, 51, 38, 45, 63, 51, 38, 47, 47,
       51, 38, 47, 27, 21, 63, 33, 51, 63, 70, 63, 45, 78, 51, 48,  6, 82,
       38, 10, 10, 78, 51,  7, 38, 39, 27, 47, 78, 51, 27, 31, 51, 48,  6,
       82, 38, 10, 10, 78, 51, 27,  6, 51, 27, 72, 31, 51, 58, 19,  6, 30,
       19, 38, 78, 52, 30, 30, 41, 70, 63, 45, 78, 72, 82, 27,  6])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [10]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [11]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)
print(one_hot.shape)

### not for our vocab containing 83 unique charcters, so our one hot encode size will be 83 , instead of 8 above.

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]
(1, 3, 8)


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [12]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    ## number of characters in one minibatch= batch_size* seq_len
    
    #batch size can be thought of as number of rown in input data format.
    
    mini_batch_size_totalchar=batch_size*seq_length
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr) // mini_batch_size_totalchar # here logic is total num of char/no of char in one minibatch
                                                      # note that we perform integer division here.
                                                      # integer div removes and extra characters.
    
    ## TODO: Keep only enough characters to make full batches
    
    # means take all the characters upto n_batches * mini_batch_size_totalchar
    # so here start index=0, and end_index=n_batches * mini_batch_size_totalchar
    
    arr = arr[:n_batches * mini_batch_size_totalchar]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(batch_size,-1) #here -1 will the second dimenison to whatever size it needs to be to accomodate 
                                  #the data
    
    ## TODO: Iterate over the batches using a window of size seq_length
    
    #here arr.shape[1] means the second dimension. we visulasing new arr as batchsize*seq_length
    
    #here range(0,arr.shape[1]) means tarversing in a list of size seqlength
    
    for n in range(0, arr.shape[1], seq_length):
        
        #the features
        #here arr[:,n:n+seq_len] means we are looking at all thr rows here, means all the batches by using colon(:)
        #after comma (,) means now about columns, we are looking at column from index n to n+seq_length
        
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [13]:
#first 100 characters in the book
text[:100]


'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [14]:
# encoded for of forst 100 characters in the book
encoded[:100]

array([65, 82, 38, 10, 72, 63, 45, 51, 24, 30, 30, 30, 32, 38, 10, 10, 78,
       51,  7, 38, 39, 27, 47, 27, 63, 31, 51, 38, 45, 63, 51, 38, 47, 47,
       51, 38, 47, 27, 21, 63, 33, 51, 63, 70, 63, 45, 78, 51, 48,  6, 82,
       38, 10, 10, 78, 51,  7, 38, 39, 27, 47, 78, 51, 27, 31, 51, 48,  6,
       82, 38, 10, 10, 78, 51, 27,  6, 51, 27, 72, 31, 51, 58, 19,  6, 30,
       19, 38, 78, 52, 30, 30, 41, 70, 63, 45, 78, 72, 82, 27,  6])

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [36]:
##get_batches(arr, batch_size, seq_length)

batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [16]:

print(x,'\ndimensions of x is :',x.shape)
print('\n')
print(y,'\ndimensions of y is :',y.shape)

[[65 82 38 10 72 63 45 51 24 30 30 30 32 38 10 10 78 51  7 38 39 27 47 27
  63 31 51 38 45 63 51 38 47 47 51 38 47 27 21 63 33 51 63 70 63 45 78 51
  48  6]
 [31 58  6 51 72 82 38 72 51 38 72 72 45 38 62 72 63 71 51 82 63 45 51 38
  72 72 63  6 72 27 58  6 51 19 38 31 51 82 63 45 51 82 48 31 28 38  6 71
  52 51]
 [63  6 71 51 58 45 51 38 51  7 58 63 67 51 82 63 51 38 70 58 27 71 63 71
  51 82 27 31 51  7 38 72 82 63 45 52 51 32 63 30 47 58 58 21 63 71 51 45
  58 48]
 [31 51 72 82 63 51 62 82 27 63  7 51 72 82 58 48 77 82 51 82 27 71 71 63
   6 30 27  6 72 63 45 63 31 72 51 58  7 51 82 27 31 51 47 27  7 63 67 51
  58  7]
 [51 31 38 19 51 82 63 45 51 72 63 38 45 68 31 72 38 27  6 63 71 67 51 10
  27 72 27  7 48 47 67 51 31 19 63 63 72 51  7 38 62 63 67 30 39 27 31 63
  45 38]
 [62 48 31 31 27 58  6 51 38  6 71 51 38  6 38 47 78 31 27 31 67 51 19 38
  31 51 27  6 51 10 45 27  6 62 27 10 47 63 51 71 27 31 38 77 45 63 63 38
  28 47]
 [51 15  6  6 38 51 82 38 71 51 31 38 27 71 51 72 82 38 72

In [35]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10],'\nsize of printed tesnor is :',x[:10, :10].shape)
print('\ny\n', y[:10, :10])

x
 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]] 
size of printed tesnor is : (1, 1, 83)

y
 [[82 38 10 72 63 45 51 24 30 30]
 [58  6 51 72 82 38 72 51 38 72]
 [ 6 71 51 58 45 51 38 51  7 58]
 [51 72 82 63 51 62 82 27 63  7]
 [31 38 19 51 82 63 45 51 72 63]
 [48 31 31 27 58  6 51 38  6 71]
 [15  6  6 38 51 82 38 71 51 31]
 [28 47 58  6 31 21 78 52 51  3]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [18]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [19]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm=nn.LSTM(len(chars) , n_hidden , n_layers , dropout = drop_prob , batch_first=True)
        
        #drouput_layer
        
        self.dropout=nn.Dropout(drop_prob)
        
        #fully connected layer
        
        self.fc=nn.Linear(n_hidden,len(chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm,
        
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output,r_hidden=self.lstm(x,hidden)
        
        ## TODO: pass through a dropout layer
        out=self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden) # out=out.view(-1,self.n_hidden) this is no longer supported 
                                                    #and we need to write the contigous() to make it work while stacking
        
        ## TODO: put x through the fully-connected layer
        out=self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [20]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [21]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [ ]:
############## JOURNEY OF THE PREPROCESSING STEP FROM SCRATCH TO BE FINALLY PASSED INTO LSTM

with open('data/anna.txt', 'r') as f:
    text = f.read()
    
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

#get_batches() takes (arr, batch_size, seq_length) and outputs a tensor of the shape (batch_size,seq_length)
batches=get_batches(encoded,8,10)

x,y=next(batches)

#onr_hot_encode takes in arguments (arr, n_labels) 
#and produces output tensor of form (batchsize,seqlen,onehot length of input)
one_hot = one_hot_encode(x, 83)

#############  DO NOT RUN THIS CELL ( ONLY FOR UNDERSTANDING PURPOSE)

In [75]:
#chcking what shape goes into out net model lstm
batches=get_batches(encoded,8,10)

#one way to findtotal number of bathes is converting generator obj to list and then findinf its len , however
#doing this makes the generator obj int list type hence next()wont  work.
#so just to find out for once we did that found number of batches=24815 guven batchsize=8, and seqlen=10

x,y=next(batches)

print('size of x',x.size,'\tshape of x :',x.shape,'\n')
print('size of y',y.size,'\tshape of y :',y.shape,'\n')

one_hot = one_hot_encode(x, 83)

print('shape of input x:',one_hot.shape)


size of x 80 	shape of x : (8, 10) 

size of y 80 	shape of y : (8, 10) 

shape of input x: (8, 10, 83)


### Set your training hyperparameters!

In [ ]:
batch_size = 200
seq_length = 100
n_epochs =  2 # start small if you are just testing initial behavior



# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [56]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [22]:
######### sample input and shapes for the codeblock below:

char='a'
h=net.init_hidden(1)
top_k=4

x = np.array([[net.char2int[char]]])

print(x)

x = one_hot_encode(x, len(net.chars))

print(x.shape)

inputs = torch.from_numpy(x)
        
if(train_on_gpu):
    inputs = inputs.cuda()
        
# detach hidden state from history
#h = tuple([each.data for each in h])

# get the output of the model
out, h = net(inputs, h)

print(out.shape)

# get the character probabilities
p = F.softmax(out, dim=1).data
if(train_on_gpu):
    p = p.cpu() # move to cpu
        
# get top characters
if top_k is None:
    top_ch = np.arange(len(net.chars))
else:
    p, top_ch = p.topk(top_k)
    print(p)
    print(p.shape)
    print(top_ch)
    print(top_ch.shape)
    top_ch = top_ch.numpy().squeeze()
    print(top_ch)
    print(top_ch.shape)
    
# select the likely next character with some element of randomness

p = p.numpy().squeeze()

print(p)
print(p.shape)

char = np.random.choice(top_ch, p=p/p.sum())
print(char)

# return the encoded value of the predicted char and the hidden state
print(net.int2char[char])

[[38]]
(1, 1, 83)
torch.Size([1, 83])
tensor([[0.0126, 0.0126, 0.0125, 0.0125]])
torch.Size([1, 4])
tensor([[13,  9, 61, 45]])
torch.Size([1, 4])
[13  9 61 45]
(4,)
[0.01263686 0.01258768 0.0125287  0.01250418]
(4,)
13
B


In [23]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [24]:
def sample(net, size, prime='love', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
        
    #use of ''.join(chars)
    #myTuple = ("John", "Peter", "Vicky")
    #x = "#".join(myTuple)
    #print(x) --------> John#Peter#Vicky
    #since we use an empty string so our out as per above exampl would be ------> JohnPeterVicky

    return ''.join(chars)

In [25]:
torch.save(model.state_dict(), PATH)

device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

NameError: name 'model' is not defined

In [ ]:
import torch
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f,map_location='cpu')
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [41]:
print(sample(loaded, 1000, prime='The', top_k=6))

Thelubury, th t
cobumerigrend obluserit weshan ore h chshe h, oure h, we h, Tce h cond, th, hes wind h h chithesth, obfrigro ch cestoure thelingaly chselous
ths hifise corid highe wobloble Ton thith, obrintonga watery th wathingero wh th ond Twe heland on onth ofiths
Cin
Tons we th to wit Tobrid, ch wereshenthe we ch ouron on
wrofuberatheshalitan
chathe
h, wingon Tate wan thery The
Cobuthagous cate
thatha cithath, t
wh oburonghshe Torery crathins
he on ths
Ca wath, che Th Torifure obus
Crifrifis ths
Toubles
h obure thate
Ths cathe
Tore orathe wathe o Tha Ths
ch Ton herere cory on th tobure h, orore chagon Tofle ha wathe wrerath,"..
henserere
Coratero hery he h cha ch hs Tcatheroberathen
chs
touth caly te hathengrobres
cobla chathingerath con Tchage cre whery hs.
Chelobrongo wifutery con chsths. cobfry
hs
thin
h ths
terofus orurigh hagh, ofushisesthe helantha cathsh, t The
hare
Tarithe wan
Trate Tcoraraths
Ch hshe Tran To wore withe
thes...", that h hely. Tcofigous wathe obrure hifry he

## Loading a checkpoint

In [44]:
import torch
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f,map_location='cpu')
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [48]:
pytorch_total_params = sum(p.numel() for p in loaded.parameters())
print(pytorch_total_params)

pytorch_total_params_trainable = sum(p.numel() for p in loaded.parameters() if p.requires_grad)
print(pytorch_total_params_trainable)

5467731
5467731


In [38]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
cobrathathista therithely Th he hes...", che
cond."
Therobrond
ths ha ths che oblonth The Th Thagarigary, hs thifuroure hery oringe Th cre
th thesthelisthe
charousthe he obrish Trigitonshe he
coront Twatobfifush crishes haginsthere Ta Th Twe
cishiganthatheshath he thigarin chins hifus thantes."
thifitory thely hatha hshagorere ougofus Thes hangofifury
hin han thath hin he cons ths cobflathatherorons The hsh Tathin Torongh hs crindelata Tre he
Tha Tarifugint cory thelifugere h h coburorobris
cry
Ths
he
ca thitorat Th Tongofuriste con cry here oury hst hingobelithelifurath ths.
chinths h hathagheshe
h he ha th ofinth heringhate he tha he hithithelagatharathind Trare
hathshs. Trond he h hathe onthere th courerigoblin thite Tagheritatherery thin hsth Tatangifus
Ths harera con orin Traths hs
Therofus. ofury," obure tharath hsthe terinth cha hififus.. ofind. this
Tore
h ofures
h cathe orate on hs
he ontath Ton Torathind.. hs
the obusthe higifligifus Twatha orery crathathige th